# All imports necessary

In [ ]:
import sys

In [ ]:
sys.path.append('..')

In [ ]:
from source.code.models.songfrequencybasedrecommender import SongFrequencyBasedRecommender

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# Read the data

## Song extra info

In [ ]:
data_directory = '../data/datasets/'

In [ ]:
song_extra_info = pd.read_csv(os.path.join(data_directory, 'song_extra_info.csv'))

In [ ]:
song_extra_info.head()

In [ ]:
song_extra_info.info(verbose=True, null_counts=True)

## Train

In [ ]:
train = pd.read_csv(os.path.join(data_directory, 'train.csv'), engine='python')

In [ ]:
train.head()

In [ ]:
train.info(verbose=True, null_counts=True)

In [ ]:
train['data_from'] = 'train'

In [ ]:
train.msno.nunique()

In [ ]:
train.song_id.nunique()

In [ ]:
train.source_system_tab.nunique()

In [ ]:
train.source_system_tab.value_counts()

In [ ]:
train.source_screen_name.nunique()

In [ ]:
train.source_screen_name.value_counts()

In [ ]:
train.source_type.nunique()

In [ ]:
train.source_type.value_counts()

In [ ]:
train.fillna('unknown', inplace=True)

In [ ]:
train.info(verbose=True, null_counts=True)

## Test

In [ ]:
test = pd.read_csv(os.path.join(data_directory, 'test.csv'), engine='python', index_col=0)

In [ ]:
test.head()

In [ ]:
test.info(verbose=True, null_counts=True)

In [ ]:
test['target'] = None

In [ ]:
test['data_from'] = 'test'

In [ ]:
test.msno.nunique()

In [ ]:
test.song_id.nunique()

In [ ]:
test.source_system_tab.nunique()

In [ ]:
test.source_system_tab.value_counts()

In [ ]:
test.source_screen_name.nunique()

In [ ]:
test.source_screen_name.value_counts()

In [ ]:
test.source_type.nunique()

In [ ]:
test.source_type.value_counts()

In [ ]:
test.fillna('unknown', inplace=True)

In [ ]:
test.info(verbose=True, null_counts=True)

## Songs

In [ ]:
songs = pd.read_csv(os.path.join(data_directory, 'songs.csv'))

In [ ]:
songs.head()

In [ ]:
songs.info(verbose=True, null_counts=True)

In [ ]:
songs.lyricist.fillna('unknown', inplace=True)

In [ ]:
songs.composer.fillna('unknown', inplace=True)

In [ ]:
songs.language.fillna(-1, inplace=True)

In [ ]:
songs.language = songs.language.astype(np.int64)

In [ ]:
songs.head()

In [ ]:
songs.info(verbose=True, null_counts=True)

In [ ]:
songs.song_length.nunique()

In [ ]:
songs.genre_ids.nunique()

In [ ]:
songs.artist_name.nunique()

In [ ]:
songs.composer.nunique()

In [ ]:
songs.lyricist.nunique()

In [ ]:
songs.language.nunique()

## Members

In [ ]:
members = pd.read_csv(os.path.join(data_directory, 'members.csv'))

In [ ]:
members.head()

In [ ]:
members.info(verbose=True, null_counts=True)

In [ ]:
members.fillna('unknown', inplace=True)

In [ ]:
members.info(verbose=True, null_counts=True)

In [ ]:
members.head()

## Sample submission

In [ ]:
sample_submission = pd.read_csv(os.path.join(data_directory, 'sample_submission.csv'))

In [ ]:
sample_submission.head(10)

In [ ]:
sample_submission.info(verbose=True, null_counts=True)

# Solutions

In [ ]:
X, y = train[train.columns[:-2]], train[train.columns[-2]]

In [ ]:
X_for_submission = test[test.columns[:-2]]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_for_submission.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.33,
    random_state=42,
    stratify=y
)

## Naive frequency approach

In [ ]:
naive_frequency_recommender = SongFrequencyBasedRecommender()

In [ ]:
print('\t', cross_val_score(naive_frequency_recommender, X_train, y_train, cv=5))

In [ ]:
naive_frequency_recommender.fit(X_train, y_train)

In [ ]:
y_pred = naive_frequency_recommender.predict(X_test)

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred)

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

## FP-GROWTH

In [ ]:
user2index = dict(zip(X_train.msno.unique(), range(len(X_train.msno.unique()))))

In [ ]:
song2index = dict(zip(X_train.song_id.unique(), range(len(X_train.song_id.unique()))))

In [ ]:
index2user = dict(zip(range(len(X_train.msno.unique())), X_train.msno.unique()))

In [ ]:
index2song = dict(zip(range(len(X_train.song_id.unique())), X_train.song_id.unique()))

In [ ]:
baskets = pd.DataFrame({'msno': X_train.msno.map(user2index), 'song_id': X_train.song_id.map(song2index)})

In [ ]:
baskets.head()

In [ ]:
baskets = baskets.groupby('msno')['song_id'].aggregate(list).reset_index()

In [ ]:
baskets.head()

In [ ]:
import pyfpgrowth
import sys
sys.setrecursionlimit(100000)

In [ ]:
patterns = pyfpgrowth.find_frequent_patterns(baskets['song_id'].values, 2)

In [ ]:
rules = pyfpgrowth.generate_association_rules(patterns, 0.1)

## SVD